# Advanced Lane Finding Project

The goals / steps of this project are the following:

- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").
- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.
- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Camera Calibration
The code for this step is contained in the 2nd code cell of the IPython notebook located in "./Trial_final_images.ipynb".

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image. Thus, objp is just a replicated array of coordinates, and objpoints will be appended with a copy of it every time I successfully detect all chessboard corners in a test image. imgpoints will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.

I then used the output objpoints and imgpoints to compute the camera calibration and distortion coefficients using the cv2.calibrateCamera() function. I applied this distortion correction to the test image using the cv2.undistort() function and obtained this result:

<img src="camera_cal/calibration1.jpg"> <img src="camera_cal/undistorted0.jpg">

## Distortion Correction

By using cv2.undistort() function with camera matrix (mtx) and distortion coefficients (dist) I transformed the test images to compensate lens distortion. Here is the code related to it:

<img src="test_images/undistorted0.jpg">

## Thresholded Binary Image
I used combination of gradx, grady, color thresholding with HSV channels, and magnitude thresholding with region of interest for all the thresholding techniques to get the resultant Binary images which are saved in output_images folder. I used region of interest because when other cars were overtaking my lane detection would also detect the lanes from the shiny doors of cars reflecting the lanes. That was not affecting the detection so much but it would have a little influence at some places where there were shadows. I could have controlled it with adjusting thresholds but then it was taking lot of time so I decided I use region of interest to resolve this issue. As you can see from the code below which is a snippet from the Trial_final_images.ipynb 6th cell. The respective functions needed for the execution are defined in the previous cell i.e. 5th cell. Choosing HSV range gave better results when compared to HLS or RGB. Adjusting V values was key because drastic changes in that range resulted in polar opposite outputs. The following is an example(threshold_image_test1.jpg) of Thresholded Binary Images from output_images folder

<img src="output_images/threshold_image_test1.jpg">

## Birds-Eye View

The perspective transform code can be found in the same cell as above and just below the above code which is in Trial_final_images.ipynb 6th cell. We define how to get the perspective transform with the help of source points(src), destination points(dst) and getPerspective() function which is saved in variable M. Then we use warpPerspective() function to get the desired persepctive transoform of the preprocess_image and M which is saved in warped variable. I hard coded the src and dst points and I tried many combinations before I could get acceptable output. This may not be the idle way to do it but due to time constraints I took this approach but soon I will be improving it by using hough lines method and evaluating 4 points out of it for each frame so that whenever the car changes lanes it's robust enough to adapt to the changes and able to perspective transform the frames to give a better output even though the surroundings are changing while changing lanes. The following cell has src, dst points, code snippet and an example of the Bird's eye view image. All the images are saved in output_images folder and it can also be viewd in the last cells of the Trial_final_images.ipynb file. Once we see the images they seem little detecting more other lanes too and that's why results in little jittery outputs. However it doesn't do a bad job at the end.

<img src="output_images/warped_image_test0.jpg">

Later we use window sliding technique to get the pixels detected and fit a line and fill the area between those lines to represent the detected road. The window sliding technique can be executed in many ways I chose to do it by finding centroids of the detected pixels with set window height and width to fill those. This code can be found in tracker.py. I import that tracker into the Trial_final_images.ipynb file and save the centroids in the window_centroids via curve_centres variable. To plot the pixels with the help of mask on both left lane and right lane using l_mask anf r_mask we input window height and width along with window centroids we detected by using tracker. This let's us plot the windows and then we segregate points detected from these windows which are pixels into left points and right points. The whole recipe is ready for detecting the pixels and plotting it on warped image. The code pertaining to this section is available in the same Jupyter notebook and is in the same cell of perspective transform which is 6th cell. It's just after the perspective transform code. Now the filling part and finding lane curvature is left. The images are in the output_images

Note: The green mask on the white lane patches is in the background and the white patches are in the foreground

<img src="output_images/lane_image_test0.jpg">

## Lane Curvature
For the lane curvature and filling the lanes we evaluate f(y) = Ay^2 + By + C for both right and left lanes. This is done by fitting the respective window centroids we found earlier and res_yvals(which is y values bottom, centroid and top of the windows we created with certain height and width after detecting pixles.) and numpy's polyfit function. After fitting this we get 3 constants as the out which become the A, B and C in the f(y) equation to get a proper equation of the line. These are done for both the lanes. Below is a small snippet of the code which does this from Trial_final_images.ipynb.

<img src="output_images/road_image_test0.jpg">

## Warping Back

We warp back the whole warped images with the lane detection onto the original road images. Along with warping back we also calculate the radius of curvature and off center parameters and put the text on the respective images. Below is the code snippet for unwarping and finding the radius of curvature. I did face a little problem while calculating radius of curvature but it's due to the fact that my lane detection isn't that great in perspective transform and that reflects in the end result but soon many things in this code will be fixed. All the code can be found in the Jupyter notebook for this project. All the rest of the images can be found in outpput_images folder

<img src="output_images/final_image_test0.jpg">

## Video

The video pipeline can be found at Trials_final_video.ipynb and the video is named under Output_processed.mp4

<video controls src="Output_processed.mp4" />

## Discussion

1) I faced few problems in deciding the src and dst points. I wanted to do a dynamic evaluation of src and dst points using Hough lines but I have kept that under my improvements list. So doing the challenge videos will be difficult with just this kind of setting.

2) I had few unsuccessful trials to do it without the tracker or a direct sliding window method. But the out wasn't fine tuned even as this one. The results for images were very good infact they were even better than the present model's image results. The video results weren't that great and that was due to not able to store properly the previous frame points.

3) My present video results do have some jitters and in few frames there are few more flickers. But the detection isn't all over the place the flickering stays in the considerable check and that's due to region of interest and adjusting the V and S threshold.

4) The model was sensitive to the adjustments done in the V thresholds. However, changes in S thresholds did change lot of things but to me V threshold changes were surprise and had tweak it very carefully.

5) Working on tracker was very enriching experience. And the Udacity's videos helped a lot in that. Doing that whole part gave me deeper insights on things can modeled in better way specifically how you track things and had intuitive understanding of it after doing this project.

6) I feel when changing lanes with my model there might be some issues of lane going haywire in that fraction of seconds when chaning lane. And if a white car comes in our lane then there are chances of it influencing our lane detection model but due to using of histogram method in tracker and adding up of all the window pixels, it's occurrence is rare.

7) In coming days I will be working on it's robustness with dynamic adaptability of src points and work on challenge videos.